In [1]:
from project.models import Project, Emprise, Plan, PlanEmprise
from public_data.models import CommunesSybarval as Comm
from django.contrib.gis.db.models import Union
from django.contrib.gis.db.models.functions import Intersection, Area, Transform
from users.models import User
from django.conf import settings
from django.core.serializers import serialize
from prettytable import PrettyTable
from django.db.models import F

default_user = User.objects.get(email__startswith="swann")

In [2]:
# show available project to let you choose an id
x = PrettyTable(field_names=["id", "Nom du projet"], align="l")
x.add_rows([(p.id, p.name) for p in Project.objects.all().order_by("id")])
print(x)

+----+------------------+
| id | Nom du projet    |
+----+------------------+
| 1  | Eyre             |
| 2  | COBAS            |
| 3  | Test geo         |
| 4  | COBAS_SHAPE      |
| 5  | Vallée de l'Eyre |
| 6  | COBAN            |
+----+------------------+


In [3]:
# get project and combined emprise
test_project = Project.objects.get(pk=2)
print(f"Project: {test_project.name}")
print("=========================")
geom = test_project.combined_emprise

test_filters = ["coveredby", "within", "contained", "crosses", "disjoint", "intersects", "overlaps"]
# contained : boss avec le rectangle englobant

communes = {_._commune: [] for _ in Comm.objects.all()}

for t in test_filters:
    for city_name in communes.keys():
        communes[city_name].append("")
    for city in Comm.objects.filter(**{f"mpoly__{t}": geom}):
        communes[city._commune][-1] = "OUI"

x = PrettyTable(field_names=["Commune"]+test_filters, align="l")
for name, row in communes.items():
    x.add_row([name]+row)
print(x)

Project: COBAS
+--------------------+-----------+--------+-----------+---------+----------+------------+----------+
| Commune            | coveredby | within | contained | crosses | disjoint | intersects | overlaps |
+--------------------+-----------+--------+-----------+---------+----------+------------+----------+
| Lugos              |           |        |           |         | OUI      |            |          |
| Belin-Beliet       |           |        |           |         | OUI      |            |          |
| Salles             |           |        |           |         |          | OUI        |          |
| Arcachon           | OUI       | OUI    | OUI       |         |          | OUI        |          |
| Lanton             |           |        |           |         | OUI      |            |          |
| Le Teich           | OUI       | OUI    | OUI       |         |          | OUI        |          |
| Audenge            |           |        |           |         | OUI      |

In [4]:
from rest_framework_gis.serializers import GeoFeatureModelSerializer, GeometryField
from rest_framework import serializers

class CustomSerializer(GeoFeatureModelSerializer):
    # intersection_area = serializers.FloatField()
    intersection = GeometryField()
    
    class Meta:
        model = Comm
        geo_field = "intersection"
        fields=('_commune', )

test_project = Project.objects.get(pk=4)
geom = test_project.combined_emprise
qs = Comm.objects.filter(mpoly__intersects=geom)
qs = qs.annotate(intersection=Intersection("mpoly", geom))
qs = qs.annotate(intersection_area=Area("intersection"))
qs = qs.annotate(area=Area("mpoly"))
for city in qs:
    print(city._commune, city.area /  1000000, city.intersection_area)
    
serializer = CustomSerializer(qs, many=True)
    
data = serializer.data
data

Salles 1.5660665856949803e-08 sq_m 8.05653652755894e-09 sq_m
Lanton 1.5654680470598874e-08 sq_m 0.015654659292237763 sq_m
Le Teich 9.744691378456302e-09 sq_m 8.698104744502601e-09 sq_m
Audenge 9.304293249840708e-09 sq_m 0.009304281387103524 sq_m
Marcheprime 2.8004875819098088e-09 sq_m 0.0028004830963408163 sq_m
Andernos-les-Bains 2.3337656931683916e-09 sq_m 0.0023337577556994575 sq_m
Ares 5.581772889406789e-09 sq_m 0.005581765724582393 sq_m
Le Barp 1.2185965966874574e-08 sq_m 1.2972613594094556e-08 sq_m
Lege-Cap-Ferret 1.0863934937190892e-08 sq_m 0.010863883111731108 sq_m
Mios 1.557048535973245e-08 sq_m 0.015570434238373771 sq_m
Biganos 5.805485982194225e-09 sq_m 0.005805473428341515 sq_m


OrderedDict([('type', 'FeatureCollection'),
             ('features',
              [OrderedDict([('type', 'Feature'),
                            ('geometry',
                             GeoJsonDict([('type', 'MultiPolygon'),
                                          ('coordinates',
                                           [[[[-1.011177286921795,
                                               44.51025181672457],
                                              [-1.01108056294645,
                                               44.5103265447937],
                                              [-1.01014527789235,
                                               44.5109031986189],
                                              [-1.00740533510309,
                                               44.5125162382228],
                                              [-1.00481717855975,
                                               44.5140335542812],
                                              [-1.00

In [5]:
from ipyleaflet import Map, GeoJSON
from rest_framework_gis.serializers import GeoFeatureModelSerializer
from django.contrib.gis.db.models.functions import Intersection, Area, Transform
from django.conf import settings
# from my django app
from public_data.models import CommunesSybarval as Comm
from project.models import Project, Emprise, Plan, PlanEmprise

class CustomSerializer(GeoFeatureModelSerializer):
    intersection = GeometryField()
    
    class Meta:
        model = Comm
        geo_field = "intersection"
        fields=('_commune', )

# get the data from the database
geom = Project.objects.get(pk=4).combined_emprise
qs = Comm.objects.filter(mpoly__intersects=geom).annotate(intersection=Intersection("mpoly", geom))
qs = qs.annotate(intersection_area=Area("intersection")).annotate(area=Area("mpoly"))
    
serializer = CustomSerializer(qs, many=True)

# Start map renderint
m = Map(center=(44.6586, -1.164), zoom=11)

geo_json = GeoJSON(
    data=serializer.data,
    style={'opacity': 1, 'dashArray': '9', 'fillOpacity': 0.1, 'weight': 1},
    hover_style={'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5},
    style_callback={'color': 'black',
                    'fillColor': random.choice(['red', 'yellow', 'green', 'orange']),}
)
m.add_layer(geo_json)
m

Map(center=[44.6586, -1.164], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

In [6]:
# lambert 93 - EPSG 2154
mios = Comm.objects.get(_commune="Mios")

def get_km_sq(mpoly): 
    """ 
    Returns the area in acres. 
    """ 
    # Convert our geographic polygons (in WGS84)
    # into a local projection for France (here lambert 93 - EPSG 2154) 
    mpoly.transform(2154)
    # we are in m² and we want km²
    return mpoly.area / (1000 * 1000)

print(get_km_sq(mios.mpoly))

137.21781664992756


In [7]:
test_project = Project.objects.get(pk=4)
geom = test_project.combined_emprise
qs = Comm.objects.filter(mpoly__intersects=geom)
qs = qs.annotate(mpoly_2=Transform("mpoly", 2154))
# qs = qs.annotate(intersection=Intersection("mpoly", geom))
# qs = qs.annotate(intersection2=Transform("intersection", 2154))
qs = qs.annotate(intersection=Transform(Intersection("mpoly", geom), 2154))
qs = qs.annotate(intersection_area=Area("intersection"))
qs = qs.annotate(area=Area("mpoly_2"))

x = PrettyTable()
x.field_names = ["Commune", "Aire (km²)", "Aire (km²) intersection COBAS", ]
x.align = "r"
x.align["Commune"] = "l"
for city in qs:
    x.add_row([city._commune, f"{city.area.sq_km:.2f}", f"{city.intersection_area.sq_km:.2f}"])
print(x)

+--------------------+------------+-------------------------------+
| Commune            | Aire (km²) | Aire (km²) intersection COBAS |
+--------------------+------------+-------------------------------+
| Salles             |     138.20 |                          0.00 |
| Lanton             |     137.57 |                        137.57 |
| Le Teich           |      85.94 |                          0.00 |
| Audenge            |      81.85 |                         81.85 |
| Marcheprime        |      24.64 |                         24.64 |
| Andernos-les-Bains |      20.52 |                         20.52 |
| Ares               |      49.03 |                         49.03 |
| Le Barp            |     107.37 |                          0.00 |
| Lege-Cap-Ferret    |      95.49 |                         95.49 |
| Mios               |     137.22 |                        137.22 |
| Biganos            |      51.12 |                         51.12 |
+--------------------+------------+-------------

In [8]:
qs = qs.filter(intersection_area__gt=F("area") * 0.95 )

x = PrettyTable()
x.field_names = ["Commune", "Aire (km²)", "Aire (km²) intersection COBAS", ]
x.align = "r"
x.align["Commune"] = "l"
for city in qs:
    x.add_row([city._commune, f"{city.area.sq_km:.2f}", f"{city.intersection_area.sq_km:.2f}"])
print(x)

+--------------------+------------+-------------------------------+
| Commune            | Aire (km²) | Aire (km²) intersection COBAS |
+--------------------+------------+-------------------------------+
| Lanton             |     137.57 |                        137.57 |
| Audenge            |      81.85 |                         81.85 |
| Marcheprime        |      24.64 |                         24.64 |
| Andernos-les-Bains |      20.52 |                         20.52 |
| Ares               |      49.03 |                         49.03 |
| Lege-Cap-Ferret    |      95.49 |                         95.49 |
| Mios               |     137.22 |                        137.22 |
| Biganos            |      51.12 |                         51.12 |
+--------------------+------------+-------------------------------+


In [9]:
from django.db import connection
qs = Comm.objects.filter(code_insee__in=["33527", ])
qs = qs.aggregate(mpoly=Union("mpoly"))
connection.queries.pop()

{'sql': 'SELECT ST_Union("public_data_communessybarval"."mpoly")::bytea AS "mpoly" FROM "public_data_communessybarval" WHERE "public_data_communessybarval"."code_insee" IN (\'33527\')',
 'time': '0.001'}

In [46]:
from django.contrib.gis.geos import Polygon as dj_Polygon

poly = dj_Polygon.from_bbox((43, -1, 44, 0))
poly.srid = 4326
poly_2 = dj_Polygon.from_bbox((43.25, -0.75, 43.75, -0.25))
poly_3 = dj_Polygon.from_bbox((43.4, -0.6, 43.6, -0.4))


clone = poly.transform(2154, clone=True)
clone.area / 10000

22290375463.42597

In [42]:
from ipyleaflet import Map, Polygonç

m = Map(center=(43.5, -0.5), zoom=8)

m.add_layer(Polygon(
    locations=poly.coords,
    color="green",
    fill_color="green"
))

m.add_layer(Polygon(
    locations=poly_2.coords,
    color="red",
    fill_color="red"
))

m.add_layer(Polygon(
    locations=poly_3.coords,
    color="blue",
    fill_color="blue"
))

m

Map(center=[43.5, -0.5], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…

In [50]:
from django.contrib.gis.geos import Polygon, MultiPolygon

BIG_SQUARE = MultiPolygon([
    Polygon.from_bbox((43, -1, 44, 0)),
], srid=4326)

BIG_SQUARE.transform(2154, clone=True).area / 1000 ** 2

22290.375463425968